# magnETHical

This notebook is a small demonstration on how to use the Python interface for the spectrometer. The following cells will connect to the spectrometer, record a free induction decay (FID) and do some simple preliminary analysis. The data is stored in `nmrpipe` format, thus further analysis can be conducted in any other tooling capable of reading that format. For more features and further information, see the various record `scripts` and the library reference.

In [1]:
%matplotlib widget

# Standard science imports
import numpy as np
import matplotlib.pyplot as plt
import nmrglue as ng

# Spectrometer imports
from spectrometer import (
    NMRSequence,
    Spectrometer,
    ConnectionSettings,
    Pulse,
    Delay,
    Record,
)
from spectrometer import FID1D

Did not find any configuration file. Using defaults.


## Setup the spectrometer

First step is to setup the spectrometer. The basic steps are 
1. Compiling and flashing the FPGA firmware
2. Running the MaRCoS server
3. Executing the python self tests

After performing these steps, come back here enter the correct connection settings.

You also need to know the transmit frequency. This should be as close as possible to the resonant frequency of the target nuclei and the magnetic field strength (and thus the temperature). Enter it together with the sample rate below as well.

In [2]:
# Setup the spectrometer, port and frequency
connection_settings = ConnectionSettings(ip_address="192.168.1.100", port=11111)

# Create the spectrometer object and connect to the hardware
spectrometer = Spectrometer(
    tx_freq=25_090_230,
    rx_freq=None,
    sample_rate=320000,
    server_config=connection_settings,
)

# Flash the FPGA bitstream
spectrometer.setup_fpga()

# Compile the server on the spectrometer
spectrometer.setup_server()

# Start the server on the spectrometer
spectrometer.start_server()

# Connect to the spectrometer server
spectrometer.connect()

MaRCoS server is already running! Stopping server...


## Perform the experiment
Recording a simple FID and a classic pulse echo sequence is straight forward since these sequences are already defined. Below we start off with sending a single pulse and recording the received signal after a delay of 30us after the end of the pulse.

In [3]:
# Define a sequence
# Here we use a 9us pulse (hoping it to be close to a 90-degree pulse) and a delay of 30us before recording (which should get rid of the coil ringing)
# For saving the data we assume that we're recording a 1H Water FID

seq_simple = NMRSequence.simple(pulse_length_us=9, delay_us=30, record_length_us=10_000)
# same as
seq_built = NMRSequence.build(
    [Pulse(duration_us=9, power=1), Delay(duration_us=30), Record(duration_us=10_000)]
)
# or
seq_manual = NMRSequence(
    tx_sequence=(np.array([0, 9]), np.array([1, 0])), rx_sequence=np.array([39, 10039])
)

assert seq_simple == seq_built == seq_manual

In [4]:
# Send the sequence
data = spectrometer.send_sequence(sequence=seq_simple)

# Save the data
fid = FID1D(
    data=data,
    spectral_width=spectrometer.sample_rate,
    carrier_freq=0.0,
    observation_freq=spectrometer.rx_freq,
    label="1H",
    sample="Water",
    pulse_file="single_90_degree_pulse,length=9us,delay=30us",
    spectrometer="magnETHical v0.1",
)
timestr = fid.timestamp.strftime("%Y%m%d-%H%M%S")
fid.to_file(f"data/{timestr}-{fid.sample}-{fid.label}-{fid.pulse_file}.fid")

TypeError: Cannot interpret '2' as a data type

In [ ]:
# Take a quick look at the time domain data (i.e. the FID)
fid.show_plot()

In [ ]:
# Take a quick look at the spectrum (includes an auto zero-fill, complex fft and auto phase correction)
fid.show_simple_fft()

# Now figure out the phase shift
# 
Probably nice to have:
- simple processing:
  - automatically cut spectrum around the peak ("extract peak" or so)
  - do a lorentz fit (decay for t2, decaying sin^2 for rabi nutation)
  - do a SNR estimation
  
Use NMRpy for that? https://github.com/NMRPy/nmrpy

In [ ]:
# Stop the server
spectrometer.stop_server()